In [ ]:
!pip install transformers streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip install accelerate
!pip install pyngrok
!ngrok authtoken 2mOqhQcSnbbYhkfCMidP4Mqrx8e_21rYHbQprP4HVABgcXhky


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import threading
import time
import subprocess
import os
import sys
from pyngrok import ngrok
import requests


def kill_existing_processes():
    """
    Kill existing ngrok and Streamlit processes
    """
    try:
        # Kill ngrok tunnels
        ngrok.kill()

        # Kill existing Streamlit processes (if any)
        if sys.platform.startswith('win'):
            os.system('taskkill /F /IM streamlit.exe')
        else:
            os.system('pkill streamlit')
    except Exception as e:
        print(f"Error killing existing processes: {e}")

def check_streamlit_running():
    """
    Check if Streamlit is running on the default port
    """
    try:
        response = requests.get('http://localhost:8501')
        return response.status_code == 200
    except:
        return False

def run_app():
    """
    Run Streamlit app
    """
    try:
        subprocess.run(["streamlit", "run", "app.py"], check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running Streamlit app: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

def start_ngrok_tunnel():
    """
    Start ngrok tunnel and return public URL
    """
    try:
        # Ensure no existing tunnels
        ngrok.kill()

        # Connect to Streamlit's default port
        public_url = ngrok.connect(addr="8501", proto="http")
        print(f"🌐 Public URL: {public_url}")
        print(f"🔐 Public Tunnel: {ngrok.get_tunnels()[0].public_url}")

        return public_url
    except Exception as e:
        print(f"Error creating ngrok tunnel: {e}")
        return None

def main():
    # Đảm bảo không có process cũ
    kill_existing_processes()

    # Chờ một chút để giải phóng port
    time.sleep(2)

    # Bắt đầu Streamlit trong một luồng riêng
    streamlit_thread = threading.Thread(target=run_app, daemon=True)
    streamlit_thread.start()

    # Chờ Streamlit khởi động
    print("🚀 Đang khởi động Streamlit...")
    time.sleep(5)

    # Kiểm tra Streamlit đã chạy chưa
    if not check_streamlit_running():
        print("❌ Streamlit không thể khởi động. Vui lòng kiểm tra app.py")
        return

    # Tạo đường hầm ngrok
    tunnel_url = start_ngrok_tunnel()

    if tunnel_url:
        print("\n🌟 Ứng dụng đã sẵn sàng!")
        print("Nhấn Ctrl+C để thoát.")

        # Giữ cho chương trình chạy
        try:
            streamlit_thread.join()
        except KeyboardInterrupt:
            print("\n🛑 Đang dừng ứng dụng...")
            ngrok.kill()

if __name__ == "__main__":
    main()

🚀 Đang khởi động Streamlit...
🌐 Public URL: NgrokTunnel: "https://15e0-34-125-110-20.ngrok-free.app" -> "http://localhost:8501"
🔐 Public Tunnel: https://15e0-34-125-110-20.ngrok-free.app

🌟 Ứng dụng đã sẵn sàng!
Nhấn Ctrl+C để thoát.
